In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os

In [3]:
df = pd.read_csv("../Data/raw/petroleum_raw.csv", delimiter=',',encoding='unicode_escape')
df.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2024-09-30,R30,PADD 3,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_R30_DPG,Gulf Coast All Grades All Formulations Retail ...,2.793,$/GAL
1,2024-09-30,R10,PADD 1,EPD2D,No 2 Diesel,PTE,Retail Sales,EMD_EPD2D_PTE_R10_DPG,East Coast No 2 Diesel Retail Prices (Dollars ...,3.571,$/GAL
2,2024-09-30,R5XCA,PADD 5 EXCEPT CALIFORNIA,EPD2D,No 2 Diesel,PTE,Retail Sales,EMD_EPD2D_PTE_R5XCA_DPG,West Coast (PADD 5) Except California No 2 Die...,3.797,$/GAL
3,2024-09-30,R40,PADD 4,EPMRU,Conventional Regular Gasoline,PTE,Retail Sales,EMM_EPMRU_PTE_R40_DPG,Rocky Mountain Regular Conventional Retail Gas...,3.421,$/GAL
4,2024-09-30,R1X,PADD 1A,EPMRU,Conventional Regular Gasoline,PTE,Retail Sales,EMM_EPMRU_PTE_R1X_DPG,New England (PADD 1A) Regular Conventional Ret...,3.058,$/GAL


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   period              1000 non-null   object 
 1   duoarea             1000 non-null   object 
 2   area-name           1000 non-null   object 
 3   product             1000 non-null   object 
 4   product-name        1000 non-null   object 
 5   process             1000 non-null   object 
 6   process-name        1000 non-null   object 
 7   series              1000 non-null   object 
 8   series-description  1000 non-null   object 
 9   value               988 non-null    float64
 10  units               1000 non-null   object 
dtypes: float64(1), object(10)
memory usage: 86.1+ KB


In [5]:
#Numerical summary (Only value)
numerical_summary = df.describe()
numerical_summary

,value
count,988.000000
mean,3.695623
std,0.563167
min,2.605000
25%,3.239500
50%,3.612000
75%,4.053500
max,5.395000


In [6]:
#Categorical summary
categorical_summary = df.describe(include=['object'])
categorical_summary

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,units
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,4,29,29,14,14,1,1,310,310,1
top,2024-09-30,R5XCA,PADD 5 EXCEPT CALIFORNIA,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPMR_PTE_SMN_DPG,Minnesota Regular All Formulations Retail Gaso...,$/GAL
freq,310,46,46,97,97,1000,1000,4,4,1000


Si duoarrea y/o area-name hacen referencia a 1 estado cada unique, solo contariamos con 29 de los 50 estados que hay en USA.
Bien puede no haber información de estos datos, o ser necesario llamar más datos de la API para un mejor analisis

# Ahora vamos por las columnas

## Period

In [7]:
df['period'].value_counts()

period
2024-09-30    310
2024-09-23    310
2024-09-16    310
2024-09-09     70
Name: count, dtype: int64

Solo cuatro periodos disponibles, en este caso 310 datos de todos los casos excepto 2024-09-09
Esta columna está considerada como object, por lo que puede ser necesaria una transformación a fecha

## Duoarea y area-name
Al parecer están relacionadas, por lo que puede ser interesante compararlas a la vez

In [8]:
df['duoarea'].value_counts()

duoarea
R5XCA    46
R40      46
R20      46
R50      45
R30      45
R1Z      45
R1Y      44
NUS      44
R1X      44
R10      42
STX      39
SNY      38
YDEN     38
SCO      38
SCA      32
Y35NY    28
Y05SF    28
YMIA     28
YCLE     28
YORD     27
Y48SE    27
Y44HO    27
Y05LA    26
SMN      26
SFL      25
SMA      25
SWA      25
SOH      24
YBOS     24
Name: count, dtype: int64

Para duoarea al parecer son codigos que representan la zona

- El código R5XCA tiene un valor asociado de 46, que coincide con el área "PADD 5 EXCEPT CALIFORNIA" en la columna "area-name".
    De las posibildades que se me ocurren para descifrar esto (o darle sentido)
    R5 = PADD 5
    X = EXCEPT
    CA = CALIFORNIA

- Este patrón se repite con otros códigos y sus respectivos nombres de área.
Por ejemplo:
R50 = 45     <->    PADD 5 = 45  

Ya que esta información se repite y de manera más clara en area-name, podriamos considerar quitar esta columna

In [11]:
df['area-name'].value_counts()

area-name
PADD 5 EXCEPT CALIFORNIA    46
PADD 4                      46
PADD 2                      46
PADD 5                      45
PADD 3                      45
PADD 1C                     45
PADD 1B                     44
U.S.                        44
PADD 1A                     44
PADD 1                      42
TEXAS                       39
NEW YORK                    38
DENVER                      38
COLORADO                    38
CALIFORNIA                  32
NEW YORK CITY               28
SAN FRANCISCO               28
MIAMI                       28
CLEVELAND                   28
CHICAGO                     27
SEATTLE                     27
HOUSTON                     27
LOS ANGELES                 26
MINNESOTA                   26
FLORIDA                     25
MASSACHUSETTS               25
WASHINGTON                  25
OHIO                        24
BOSTON                      24
Name: count, dtype: int64

### Consideraciones de esta columna:

### 1. PADD (Petroleum Administration for Defense Districts):
Descripción: Los PADDs son divisiones geográficas utilizadas por la Administración de Información de Energía (EIA) en los EE. UU. para rastrear los datos de suministro y distribución de petróleo. Estas áreas fueron originalmente creadas durante la Segunda Guerra Mundial para coordinar el suministro de petróleo.

PADD 1, 1A, 1B, 1C: PADD 1 cubre la Costa Este de los EE. UU., y está subdividido en 3 áreas:
PADD 1A: Nueva Inglaterra (Maine, Vermont, New Hampshire, Massachusetts, Connecticut, Rhode Island).
PADD 1B: Centro Atlántico (Nueva York, Nueva Jersey, Pensilvania, Delaware, Maryland, Washington D.C.).
PADD 1C: Sureste (Virginia, Carolina del Norte, Carolina del Sur, Georgia, Florida).

Otros PADDs:

PADD 2: El Medio Oeste (desde Ohio hasta Dakota del Norte).
PADD 3: La Costa del Golfo (Texas, Luisiana, Mississippi, Alabama, Nuevo México).
PADD 4: Montaña (desde Wyoming hasta Montana, incluyendo Colorado).
PADD 5: Costa Oeste (California, Nevada, Oregón, Washington, Alaska, Hawái).
PADD 5 EXCEPT CALIFORNIA: Esto excluye específicamente a California dentro del PADD 5.


### 2. Estados y ciudades (Texas, New York, Denver, etc.):
Descripción: Algunas de las áreas mencionadas son estados individuales, como TEXAS, COLORADO, MINNESOTA, o OHIO, y otras son ciudades específicas, como DENVER, LOS ANGELES, CHICAGO, y HOUSTON.
Diferencias entre ciudades y estados: Por ejemplo:

New York se refiere al estado de Nueva York, mientras que New York City es la ciudad específica dentro de ese estado.
Esto es similar para otras combinaciones como SAN FRANCISCO (ciudad) y CALIFORNIA (estado).

## Product

In [12]:
df['product'].value_counts()

product
EPM0        97
EPMM        96
EPMP        94
EPMR        91
EPMMR       75
EPMRR       74
EPM0R       73
EPMMU       69
EPMPR       68
EPMPU       66
EPM0U       66
EPMRU       64
EPD2DXL0    34
EPD2D       33
Name: count, dtype: int64

In [13]:
df['product-name'].value_counts()

product-name
Total Gasoline                       97
Midgrade Gasoline                    96
Premium Gasoline                     94
Regular Gasoline                     91
Gasoline Reformulated Midgrade       75
Reformulated Regular Gasoline        74
Reformulated Motor Gasoline          73
Gasoline Conventional Midgrade       69
Reformulated Premium Gasoline        68
Conventional Premium Gasoline        66
Conventional Gasoline (No Oxy)       66
Conventional Regular Gasoline        64
No 2 Diesel Low Sulfur (0-15 ppm)    34
No 2 Diesel                          33
Name: count, dtype: int64

Similar a duoarea y area... Pero en este caso propongo algunas transformaciones.

Ya que product son codigos que repiten la información (solo que codificada) de product-name, se deberian reemplazar todos los codigos que hagan referencia a gasolina como "Gasoline"

## Process

In [14]:
df['process'].value_counts()

process
PTE    1000
Name: count, dtype: int64

In [15]:
df['process-name'].value_counts()

process-name
Retail Sales    1000
Name: count, dtype: int64

Unico valor para todos los datos, como la API puede dar para el por mayor o para productos que no sean gasolina/diesel, debe tener esta columna... Por ahora nos resulta repetitiva

## Series

In [18]:
df['series'].value_counts()

series
EMM_EPMR_PTE_SMN_DPG       4
EMM_EPMRR_PTE_R50_DPG      4
EMM_EPM0_PTE_YMIA_DPG      4
EMM_EPM0R_PTE_Y05SF_DPG    4
EMM_EPMM_PTE_SMA_DPG       4
                          ..
EMM_EPMPU_PTE_R30_DPG      3
EMM_EPMPR_PTE_YBOS_DPG     3
EMM_EPMP_PTE_Y05LA_DPG     3
EMM_EPMP_PTE_SNY_DPG       3
EMM_EPMRU_PTE_R40_DPG      3
Name: count, Length: 310, dtype: int64

In [17]:
df['series-description'].value_counts()

series-description
Minnesota Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)           4
West Coast Regular Reformulated Retail Gasoline Prices (Dollars per Gallon)              4
Miami, FL All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)        4
San Francisco, CA All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon)    4
Massachusetts Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon)      4
                                                                                        ..
Gulf Coast Premium Conventional Retail Gasoline Prices (Dollars per Gallon)              3
Boston, MA Premium Reformulated Retail Gasoline Prices (Dollars per Gallon)              3
Los Angeles Premium All Formulations Retail Gasoline Prices (Dollars per Gallon)         3
New York Premium All Formulations Retail Gasoline Prices (Dollars per Gallon)            3
Rocky Mountain Regular Conventional Retail Gasoline Prices (Dollars per

Todavia no estoy seguro si estas columnas se van o se quedan, repiten (y compilan) mucha información; pero es curioso que por ejemplo no veo ningún "Total gasoline", sino que al parecer están expresadas de otras formas. No la borraré hasta que la haya analizado un poco más... 

## Prices

In [19]:
df['value'].value_counts()

value
3.118    7
3.844    6
3.818    5
3.105    5
3.093    5
        ..
4.766    1
4.078    1
4.654    1
3.995    1
3.071    1
Name: count, Length: 589, dtype: int64

In [20]:
df['units'].value_counts()

units
$/GAL    1000
Name: count, dtype: int64

Ya sabemos que es Dolares por galón, por lo que podriamos eliminar units y cambiar el nombre de value a algo como "Value ($/GAL)" y que en la misma columna se entienda que son dolares por galones... Pero en caso de que se fueran a juntar con fuentes por fuera de USA, si seria importante guardar la columna